Llama Index Workflow Basics and Examples
- https://docs.llamaindex.ai/en/stable/examples/workflow/workflows_cookbook/
- https://github.com/run-llama/llama_index/tree/main/docs/docs/examples/workflow

In [22]:
from llama_index.core.workflow import (
    Event,
    StartEvent,
    StopEvent,
    Workflow,
    step,
    Context,
)
import random
from llama_index.utils.workflow import draw_most_recent_execution,draw_all_possible_flows
from llama_index.llms.openai import OpenAI
#from llama_index.core.workflow import draw_all_possible_flows

In [6]:
import os 
from dotenv import load_dotenv

env_path = '../../.env'
load_dotenv(dotenv_path=env_path)

openai_apikey = os.environ["OPENAI_API_KEY"] 

- Intro to functionalities 
    - most basic workflow 

In [8]:
class OpenAIGenerator(Workflow):
    @step
    async def generate(self, ev: StartEvent) -> StopEvent:
        llm = OpenAI(model="gpt-4o-mini",api_key=openai_apikey)
        response = await llm.acomplete(ev.query)
        return StopEvent(result=str(response))

In [9]:
w = OpenAIGenerator(timeout=10, verbose=False)
result = await w.run(query="What's LlamaIndex?")
print(result)

LlamaIndex, formerly known as GPT Index, is a framework designed to facilitate the integration of large language models (LLMs) with various data sources. It provides tools and structures to help developers create applications that can efficiently query and retrieve information from different types of data, such as documents, databases, and APIs, using natural language processing.

The main features of LlamaIndex include:

1. **Data Connection**: It allows users to connect to various data sources, making it easier to pull in relevant information for processing.

2. **Indexing**: LlamaIndex helps in indexing the data, which optimizes the retrieval process and enhances the performance of queries made to the LLM.

3. **Querying**: Users can perform natural language queries against the indexed data, enabling more intuitive interactions with the information.

4. **Integration with LLMs**: The framework is designed to work seamlessly with large language models, allowing developers to leverage

In [23]:
## show workflow in a diagram
temp_file_name = '/data/home/xiong/data/temp/temp_workflow.html'
draw_all_possible_flows(OpenAIGenerator,filename=temp_file_name)

/data/home/xiong/data/temp/temp_workflow.html


- Loops and branches
- Let's go to a more interesting example, demonstrating our ability to loop:

In [18]:
from typing import Union

In [19]:
class FailedEvent(Event):
    error: str
class QueryEvent(Event):
    query: str
class LoopExampleFlow(Workflow):
    @step
    async def answer_query(self, ev: Union[StartEvent,QueryEvent ]) ->  Union[FailedEvent, StopEvent]:  ## it can take start ecent or query event and emmit eigher failedevent or stopevent
        query = ev.query
        # try to answer the query
        random_number = random.randint(0, 1)
        if random_number == 0:
            return FailedEvent(error="Failed to answer the query.")
        else:
            return StopEvent(result="The answer to your query")

    @step
    async def improve_query(self, ev: FailedEvent) ->  Union[QueryEvent, StopEvent]:
        # improve the query or decide it can't be fixed
        random_number = random.randint(0, 1)
        if random_number == 0:
            return QueryEvent(query="Here's a better query.")
        else:
            return StopEvent(result="Your query can't be fixed.")

In [24]:
draw_all_possible_flows(LoopExampleFlow, filename=temp_file_name)

/data/home/xiong/data/temp/temp_workflow.html


![Alt text](workflow_loop.png)